In [183]:
import pymongo
import requests
import json
import numpy as np
import pandas as pd
import time
from tqdm import tqdm
import datetime
from credentials import client_id,client_secret

In [184]:
def cleanLinks(link):
    links = link.split(",")
#     print(links)
    clean_links=[]
    urls = ""
    rel = ""
    for i in links:
        temp = []
        urls,rel= i.split(";")
        urls = urls.strip()
        rel = rel.strip()
        urls = urls[1:-1]
        rel = rel[5:-1]
    #     print(urls)
    #     print(rel)

        temp.append(urls)
        temp.append(rel)

        clean_links.append(temp)
        
    return clean_links

In [185]:
def getOrgNRepo(org_name,repo_name,coll_name):
    OrgNRepo_client = pymongo.MongoClient("mongodb://127.0.0.1:27017/")
    repodb = OrgNRepo_client["repo_collector"]
    tf_repo = repodb[coll_name]
    repo_details = tf_repo.find({"full_name":org_name+"/"+repo_name}).limit(1).sort("$natural",-1)
    repo_id = "NA"
    org_id = "NA"
    for res in repo_details:
        repo_id = (res['id'])
        owner = res['owner']
        org_id = owner['id']
    OrgNRepo_client.close()
    return repo_id, org_id
        

In [186]:
def addMISCDetails(repo_id, org_id, list_details, isMember, isContributor):
    updt_list = []
    if isMember or isContributor:
        add_details = {"org_id":org_id, "repo_id":repo_id, "isMember": isMember, "isContributor":isContributor}
        for ele in list_details:
            ele.update(add_details)
            updt_list.append(ele)
    else:    
        add_details = {"org_id":org_id, "repo_id":repo_id}
        for ele in list_details:
            ele.update(add_details)
            updt_list.append(ele)            
    return updt_list

In [187]:
def populateToDB(url, org_name, repo_name, db_name, user):
    
    response = requests.get(url,auth=(client_id, client_secret))
    print(response.headers.get('Link'))
#     print(response.headers.get('X-RateLimit-Remaining'))
    if int(response.headers.get('X-RateLimit-Remaining')) <= 2:
        time.sleep(3600)
        
    raw_list = response.json()
#     print(len(raw_list))
    
    mongo_client = pymongo.MongoClient("mongodb://127.0.0.1:27017/")
    repodb = mongo_client["repo_collector"]
    tensorflow_coll = repodb[db_name]
    
#     print(raw_list)
    repo_id, org_id = getOrgNRepo(org_name,repo_name,"tf_repo")
#     print(str(repo_id)+" | "+str(org_id))
    isMember = isContributor = 0
    if user == "members":
        isMember = 1
    elif user == "contributors":
        isContributor = 1
    
    updt_list = addMISCDetails(repo_id, org_id, raw_list, isMember, isContributor)
    #print(issue_raw_list[-1])
    
    tensorflow_coll.insert_many(updt_list)
    
    if response.headers.get('Link'):
        link = response.headers.get('Link')
    #     print(link)
        clean_links = cleanLinks(link)        
    #     print(clean_links)

        df = pd.DataFrame(clean_links, columns =['url', 'rel'])
    #     print(df)
        start_time = time.time()
        while "next" in df['rel'].values:
    
            index = int(df[df['rel']=='next'].index.values)
#             print(df['url'][index])
            url = df['url'][index]
            response = requests.get(url,auth=(client_id, client_secret))
            raw_list = response.json()
            
            updt_list = addMISCDetails(repo_id, org_id, raw_list, isMember, isContributor)
            
            tensorflow_coll.insert_many(updt_list)
            
            print(response.headers.get('X-RateLimit-Remaining'))
            if int(response.headers.get('X-RateLimit-Remaining')) <= 2:
                time.sleep(3600)
            
            link = response.headers.get('Link')
            clean_links = cleanLinks(link)
            df = pd.DataFrame(clean_links, columns =['url', 'rel'])
            
        print(time.time() - start_time)
        
    mongo_client.close()

In [188]:
org_name = "tensorflow"
repo_name = "tensorflow"

In [189]:
issues_url = "https://api.github.com/repos/"+org_name+"/"+repo_name+"/issues?state=all&per_page=100"
# populateToDB(issues_url, org_name, repo_name, "repo_issues", "NA")

In [190]:
forks_url = "https://api.github.com/repos/"+org_name+"/"+repo_name+"/forks?per_page=100"
# populateToDB(forks_url, org_name, repo_name, "repo_forks", "NA")

In [191]:
releases_url = "https://api.github.com/repos/"+org_name+"/"+repo_name+"/releases?per_page=100"
# populateToDB(releases_url, org_name, repo_name, "repo_releases", "NA")

In [192]:
pulls_url = "https://api.github.com/repos/"+org_name+"/"+repo_name+"/pulls?per_page=100"
# populateToDB(pulls_url, org_name, repo_name, "repo_pulls", "NA")

In [193]:
members_url = "https://api.github.com/orgs/"+org_name+"/members?per_page=100"
# populateToDB(members_url, org_name, repo_name, "repo_users", "members")

In [194]:
contributors_url = "https://api.github.com/repos/"+org_name+"/"+repo_name+"/contributors?per_page=100&anon=true"
# populateToDB(contributors_url, org_name, repo_name, "repo_users", "contributors")

In [195]:
labels_url = "https://api.github.com/repos/"+org_name+"/"+repo_name+"/labels?per_page=100"
# populateToDB(labels_url, org_name, repo_name, "repo_labels", "NA")